In [34]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget
import h5py
import hdf5plugin
import correction
import os 
import numpy as np
import matplotlib.pyplot as plt
#Data Files


project_dir = '/data/visitors/balder/20220115/2022022408'
experiment_dir = os.path.join(project_dir, 'raw/XRD Eiger Test')
poni_dir = os.path.join(project_dir, 'process')

stacks = {
    "no_tilt_close": {
        "filename": 'LaB6_9_data_000001.h5',
        "poni": 'samuel_9_0.poni'
    },
    "large_tilt_close": {
        "filename": 'LaB6_10_data_000001.h5',
        "poni": 'samuel_10_0.poni'
    },
    "small_tilt_close": {
        "filename": 'LaB6_11_data_000001.h5',
        "poni": 'samuel_11_0.poni'
    },
    #"no_tilt_far": {
    #    "filename": 'LaB6_13_data_000001.h5',
    #    "poni": 'samuel_13_0.poni'
    #}
}

icsdfilepath = os.path.join(experiment_dir,'LaB6-icsd.txt')
#Parameters
pl = 75  # pixel length, in micrometers 
pw = 450 # pixel width, in micrometers
ph = 75  # pixel height, in micrometers
pixel_dimensions = (pl, pw, ph)
az_npt = 1000 #number of azimuthal integration points

def load_stack(name):
    stack = stacks[name]
    image_path = os.path.join(experiment_dir, stack["filename"])
    with h5py.File(image_path,'r') as h5f:
        data = h5f['/entry/data/data']
        data = np.squeeze(data)
    poni_path = os.path.join(poni_dir, stack["poni"])
    poni = correction.load_poni(poni_path)
    return poni, data


saves = {}
for stack_name in stacks.keys():
    poni, data = load_stack(stack_name)
    img, mask = correction.mask(data[1], False)
    #plt.figure(figsize=(15, 15))
    #plt.imshow(img, vmax=np.percentile(img, 99))
    #plt.plot(poni['Poni2']*1e6/pl,poni['Poni1']*1e6/ph,'rx')
    #plt.title(f'{stack_name} - no correction')
    #integrator, det_params_b711, tth_hkl = correction.azimutal_fit(img, poni, icsdfilepath, pixel_dimensions)
    #correction.plot_fit(img, poni, det_params_b711, tth_hkl, pixel_dimensions)
    recovered_img = correction.correct_depth_spill(img, poni, pixel_dimensions)
    #plt.figure(figsize=(15, 15))
    #plt.imshow(recovered_img, vmax=np.percentile(img, 99))
    #plt.plot(poni['Poni2']*1e6/pl,poni['Poni1']*1e6/ph,'rx')
    #plt.title(f'{stack_name} - corrected')
    saves[stack_name] = {
        "original_image": img,
        "recovered_image": recovered_img,
    }
    #correction.plot_compare_images(img, recovered_img)
    #deg_xtth, az_before = integrator.integrate1d(img, az_npt)
    #deg_xtth, az_after = integrator.integrate1d(recovered_img, az_npt)
    #correction.plot_compare_peaks(az_before, az_after, deg_xtth, tth_hkl)
    #correction.plot_compare_peak_fit(az_before, az_after, deg_xtth, tth_hkl)

perc = np.percentile(saves["no_tilt_close"]["original_image"], 99)

data_layer = 3
comparisons = {
    "stroke_comparison": {
        "No Tilt - Uncorrected": {
            "source": "no_tilt_close",
            "image": "original_image",
            "data_layer": data_layer,
            "area": (slice(0,500), slice(200,300)),
        },
        "No Tilt - Corrected": {
            "source": "no_tilt_close",
            "image": "recovered_image",
            "data_layer": data_layer,
            "area": (slice(0,500), slice(200,300)),
        },
        "Tilt - Uncorrected (close to PONI)": {
            "source": "large_tilt_close",
            "image": "original_image",
            "data_layer": data_layer,
            "area": (slice(0,500), slice(0,100)),
        },
    }
}

for comparison in comparisons.values():
    n = len(comparison)
    fig, axs = plt.subplots(1, n, figsize=(15, 15))
    for i, (name, info) in enumerate(comparison.items()):
        axs[i].imshow(saves[info["source"]][info["image"]][info["area"]], vmax=perc)
        axs[i].set_title(name)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …